In [2]:
import os
from glob import glob

import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

from ultralytics import YOLO

from keras.utils import to_categorical

# RAW Data Preparation 

In [ ]:
# Get the current working directory
basepath = os.getcwd()
print(f"Current directory path: {basepath}")

# Change the directory to the dataset folder
data_folder = os.path.join(basepath, "kaggle-satellite-imagery/shipsnet/shipsnet")
print(f"Data folder path: {data_folder}")

# Generate a list of files for ship and no_ship files
files_list_no_ship = glob(os.path.join(data_folder, "0_*"))
files_list_ship = glob(os.path.join(data_folder, "1_*"))

# Verify files are loaded
print(f"No-ship files: {len(files_list_no_ship)}, Ship files: {len(files_list_ship)}")

# Initialize image and label arrays
images = []
labels = []
text_labels = ['No ship', 'Ship']

# Load no-ship images
for file in files_list_no_ship:
    img = cv2.imread(file)
    if img is not None:  # Check if image is loaded properly
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
        labels.append(0)

# Load ship images
for file in files_list_ship:
    img = cv2.imread(file)
    if img is not None:  # Check if image is loaded properly
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
        labels.append(1)

# Convert to numpy arrays
images = np.array(images, dtype=np.int64)
labels = np.array(labels, dtype=np.int64)

# Verify dataset size
print(f"Total images: {len(images)}, Total labels: {len(labels)}")

# Split the data into train (70%), validation (20%), and test (10%) datasets
np.random.seed(27)
indices = np.arange(len(images))
np.random.shuffle(indices)

images = images[indices]
labels = labels[indices]

train = int(0.7 * len(images))
validation = int(0.2 * len(images))
test = len(images) - (train + validation)

X_train, y_train = images[:train], labels[:train]
X_validation, y_validation = images[train:(train + validation)], labels[train:(train + validation)]
X_test, y_test = images[(train + validation):], labels[(train + validation):]

# Ensure the split is correct
print(f"Train: {len(X_train)}, Validation: {len(X_validation)}, Test: {len(X_test)}")

# One-hot encode the labels
n_classes = 2
y_train = to_categorical(y_train, n_classes)
y_validation = to_categorical(y_validation, n_classes)
y_test = to_categorical(y_test, n_classes)

# Plot some sample images
if len(X_train) > 0:
    n_rows = 3
    n_cols = 5

    plt.figure(figsize=(15, 10))
    for i in range(n_rows * n_cols):
        ax = plt.subplot(n_rows, n_cols, i + 1)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        index = random.randint(0, len(X_train) - 1)
        plt.imshow(X_train[index])
        plt.title(text_labels[np.argmax(y_train[index])])
    plt.show()
else:
    print("Training dataset is empty. Please check dataset loading and splitting.")

# Data Preparation for Classify YOLO

In [3]:
# import os
# import cv2
# import numpy as np
# from datetime import datetime

# def prepare_yolo_dataset(X_train, y_train, X_validation, y_validation, X_test, y_test):
#     """
#     Prepares a dataset directory structure for YOLO classification.

#     Args:
#         X_train (numpy.ndarray): Training images.
#         y_train (numpy.ndarray): Training labels (one-hot encoded).
#         X_validation (numpy.ndarray): Validation images.
#         y_validation (numpy.ndarray): Validation labels (one-hot encoded).
#         X_test (numpy.ndarray): Test images.
#         y_test (numpy.ndarray): Test labels (one-hot encoded).
    
#     Returns:
#         str: Path to the YOLO dataset directory.
#     """
#     # Base directory for organizing the dataset
#     base_dir = "YOLO"

#     # Generate a timestamp for the folder name
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     yolo_dir = os.path.join(base_dir, timestamp)

#     # Create subdirectories for YOLO
#     train_dir = os.path.join(yolo_dir, "train")
#     val_dir = os.path.join(yolo_dir, "val")
#     test_dir = os.path.join(yolo_dir, "test")

#     # Subfolders for classes within train, val, and test
#     os.makedirs(os.path.join(train_dir, "0"), exist_ok=True)
#     os.makedirs(os.path.join(train_dir, "1"), exist_ok=True)
#     os.makedirs(os.path.join(val_dir, "0"), exist_ok=True)
#     os.makedirs(os.path.join(val_dir, "1"), exist_ok=True)
#     os.makedirs(os.path.join(test_dir, "0"), exist_ok=True)
#     os.makedirs(os.path.join(test_dir, "1"), exist_ok=True)

#     # Helper function to save images
#     def save_images(images, labels, output_subdir):
#         for i, (img, label) in enumerate(zip(images, labels)):
#             class_dir = os.path.join(output_subdir, str(label))
#             filename = f"img_{i}.jpg"
#             filepath = os.path.join(class_dir, filename)
            
#             # Convert image to uint8 format
#             if img.dtype != np.uint8:
#                 img = np.clip(img, 0, 255)  # Ensure pixel values are within range
#                 img = img.astype(np.uint8)
            
#             # Save the image
#             cv2.imwrite(filepath, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

#     # Save train, validation, and test datasets
#     save_images(X_train, np.argmax(y_train, axis=1), train_dir)
#     save_images(X_validation, np.argmax(y_validation, axis=1), val_dir)
#     save_images(X_test, np.argmax(y_test, axis=1), test_dir)

#     # Create the YOLO data.yaml file
#     yaml_content = f"""
#     path: {yolo_dir}
#     train: {os.path.abspath(train_dir)}
#     val: {os.path.abspath(val_dir)}
#     test: {os.path.abspath(test_dir)}
#     nc: 2
#     names:
#       0: No Ship
#       1: Ship
#     """

#     yaml_path = os.path.join(yolo_dir, "data.yaml")
#     with open(yaml_path, "w") as yaml_file:
#         yaml_file.write(yaml_content)

#     print(f"Dataset and YOLO configuration created successfully in: {yolo_dir}")
#     return yolo_dir


# Data preparation for Object Detection YOLO

In [4]:
from datetime import datetime

def prepare_yolo_dataset(X_train, y_train, X_validation, y_validation, X_test, y_test):
    """
    Prepares a dataset directory structure for YOLO detection.

    Args:
        X_train (numpy.ndarray): Training images.
        y_train (numpy.ndarray): Training labels (one-hot encoded).
        X_validation (numpy.ndarray): Validation images.
        y_validation (numpy.ndarray): Validation labels (one-hot encoded).
        X_test (numpy.ndarray): Test images.
        y_test (numpy.ndarray): Test labels (one-hot encoded).
    
    Returns:
        str: Path to the YOLO dataset directory.
    """
    # Base directory for organizing the dataset
    base_dir = "YOLO"

    # Generate a timestamp for the folder name
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    yolo_dir = os.path.join(base_dir, timestamp)

    # Create subdirectories for YOLO
    train_dir = os.path.join(yolo_dir, "train/images")
    val_dir = os.path.join(yolo_dir, "val/images")
    test_dir = os.path.join(yolo_dir, "test/images")
    train_labels_dir = os.path.join(yolo_dir, "train/labels")
    val_labels_dir = os.path.join(yolo_dir, "val/labels")
    test_labels_dir = os.path.join(yolo_dir, "test/labels")

    # Create directories for images and labels
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)
    os.makedirs(test_labels_dir, exist_ok=True)

    # Helper function to save images and labels
    def save_images_and_labels(images, labels, image_output_dir, label_output_dir):
        for i, (img, label) in enumerate(zip(images, labels)):
            # Save image
            image_filename = f"img_{i}.jpg"
            image_filepath = os.path.join(image_output_dir, image_filename)
            
            # Convert image to uint8 format
            if img.dtype != np.uint8:
                img = np.clip(img, 0, 255)  # Ensure pixel values are within range
                img = img.astype(np.uint8)
            
            # Save the image
            cv2.imwrite(image_filepath, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            # Save label
            label_filename = f"img_{i}.txt"
            label_filepath = os.path.join(label_output_dir, label_filename)
            with open(label_filepath, "w") as label_file:
                if np.argmax(label) == 1:  # Only create bounding box for images with ships
                    # Class 1 (Ship), bbox centered at (0.5, 0.5) with width and height of 1 (entire image)
                    label_file.write("1 0.5 0.5 1 1\n")

    # Save train, validation, and test datasets
    save_images_and_labels(X_train, y_train, train_dir, train_labels_dir)
    save_images_and_labels(X_validation, y_validation, val_dir, val_labels_dir)
    save_images_and_labels(X_test, y_test, test_dir, test_labels_dir)

    # Create the YOLO data.yaml file
    yaml_content = f"""
    path: {yolo_dir}
    train: {os.path.abspath(train_dir)}
    val: {os.path.abspath(val_dir)}
    test: {os.path.abspath(test_dir)}
    nc: 2
    names:
      0: No Ship
      1: Ship
    """

    yaml_path = os.path.join(yolo_dir, "data.yaml")
    with open(yaml_path, "w") as yaml_file:
        yaml_file.write(yaml_content)

    print(f"Dataset and YOLO configuration created successfully in: {yolo_dir}")
    return yolo_dir


# Create YOLO MODEL

In [5]:
def create_yolov8(model_type='yolov8n.pt', input_shape=(80, 80), n_classes=2):
    """
    Creates a YOLOv8 classification model.
    
    :param model_type: The type of YOLOv8 model (e.g., yolov8n-cls, yolov8s-cls, etc.)
    :param input_shape: The input shape of images (height, width).
    :param n_classes: Number of classes for classification.
    :return: Trained YOLO model ready for further use or training.
    """
    # Initialize the model
    model = YOLO(model_type)

    # Configure the model for the dataset
    model.model.input_size = input_shape
    model.model.nc = n_classes  # Number of classes

    return model

# Prepare Data and Create Model

In [ ]:
yolo_dataset_path = prepare_yolo_dataset(X_train, y_train, X_validation, y_validation, X_test, y_test)

# Initialize the YOLOv8 classification model
model = create_yolov8(model_type='yolov8n.pt', input_shape=(80, 80), n_classes=2)
print(f"Dataset prepared at: {yolo_dataset_path}")

# Train Data using information from data.yaml

In [ ]:
# Train YOLOv8 using the dataset
current_dir = os.getcwd()

the_folder = os.path.join(current_dir, yolo_dataset_path)
print(f"Current directory: {the_folder}")

yaml_data = os.path.join(the_folder, "data.yaml")
print(f"Data.yaml path: {yaml_data}")

# Train the model with verbose logging enabled
model.train(
    data=yaml_data,  # Path to the dataset's data.yaml
    epochs=3,         # Number of epochs
    batch=8,          # Image size
    imgsz=96,         # Batch size
    project=the_folder,  # Specify the folder to save the training results
    name="train_results",  # Subdirectory name under 'the_folder'
    verbose=True,
    patience=5,
)
print("YOLOv8 training completed.")

# Predict Images from Test Data

In [ ]:
# Perform inference on the test dataset defined in 'data.yaml'
results = model.predict(source=os.path.join(the_folder, 'test/images'), save=True, save_txt=True)

# Print the results (you can inspect each predicted image and bounding box information)
print("Predictions completed.")


# Plotting predicted images randomly
from save directory

In [ ]:
latest_results = results[-1]  # Get the latest results
# Print the results (you can inspect each predicted image and bounding box information)
print("Predictions completed.")
save_dir = latest_results.save_dir
print(f"Save directory: {save_dir}")

images = []

# Load the predicted images from the save directory
for img_name in os.listdir(save_dir):
    if img_name.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(save_dir, img_name)
        images.append(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))

# Randomly select 25 images for visualization
indices = random.sample(range(len(images)), min(25, len(images)))

# Plot 25 images, 5 rows, 5 columns, without labels
n_rows, n_cols = 5, 5
plt.figure(figsize=(15, 15))
for i, idx in enumerate(indices):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.imshow(images[idx])
plt.tight_layout()
plt.show()

# Extracting predictions and true labels for visualization

In [9]:
# Extracting predictions and true labels for visualization
predicted_labels = []
true_labels = []
images = []

# Load the predictions and true labels
for result in results:
    img_path = result.path
    images.append(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    
    # Extract true label from corresponding label file in 'test/labels'
    label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt')
    with open(label_path, 'r') as label_file:
        label_content = label_file.read().strip()
        if label_content:
            true_label = 1 if label_content.startswith('1') else 0
        else:
            true_label = 0
    true_labels.append(true_label)
    
    # Get predicted label
    if len(result.boxes) > 0:  # If at least one box is predicted, it's classified as 'Ship'
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)


# Plotting predicted images with label

In [ ]:

# Randomly select 15 images for visualization
indices = random.sample(range(len(images)), min(15, len(images)))

# Plot 15 images, 3 rows, 5 columns, showing predictions vs. true labels
n_rows, n_cols = 3, 5
plt.figure(figsize=(15, 10))
for i, idx in enumerate(indices):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    plt.imshow(images[idx])
    true_label = 'Ship' if true_labels[idx] == 1 else 'No Ship'
    predicted_label = 'Ship' if predicted_labels[idx] == 1 else 'No Ship'
    plt.title(f"True: {true_label}\nPred: {predicted_label}")
plt.tight_layout()
plt.show()

# Trying to use the model for real scene but not working XD

In [ ]:
# kaggle_data = os.path.join(basepath, "kaggle-satellite-imagery/scenes/scenes/lb_3.png")
kaggle_data = os.path.join(basepath, "kaggle-satellite-imagery/scenes/scenes")
print(f"Kaggle data folder path: {kaggle_data}")

# Perform inference on the test dataset defined in 'data.yaml'
results = model.predict(source=kaggle_data, imgsz=1024)
print(results)

# Display the result
# Load the image with detections
img = cv2.imread(kaggle_data)
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Get bounding box coordinates
        confidence = box.conf  # Confidence score
        class_id = int(box.cls)  # Class ID

        # Draw the bounding box
        label = f"Ship ({confidence.item():.2f})" if class_id == 1 else f"Not Ship ({confidence.item():.2f})"
        color = (0, 255, 0) if class_id == 1 else (255, 0, 0)  # Green for ship, red otherwise
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Convert image to RGB (for displaying in matplotlib)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plot the image with the detected bounding boxes
plt.figure(figsize=(10, 10))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()


# Trying to use another model manually

In [ ]:
# Load the trained YOLO model
model_test = YOLO('D:\\program\\object-detection\\YOLO\\20241119_042947\\train_results\\weights\\best.pt')  # Replace with the path to your trained model

# Path to the image you want to perform inference on
kaggle_data = os.path.join(basepath, "kaggle-satellite-imagery/scenes/scenes")

# Perform inference on the image
results_test = model_test.predict(source=kaggle_data, imgsz=1024)
